In [22]:
import pandas as pd, numpy as np

In [23]:
df = pd.read_csv("C:\\Users\\ASUS\\OneDrive\\Desktop//social.csv")

This is the dataset . Here, there is no use of the User ID feature (it is not related with any of the feature ). so we,r going to drop it. 

In [24]:
df

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


dropped User ID feuture 

In [27]:
df.drop("User ID", axis =1, inplace = True)

In [28]:
df

,Gender,Age,EstimatedSalary,Purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0
...,...,...,...,...
395,Female,46,41000,1
396,Male,51,23000,1
397,Female,50,20000,1
398,Male,36,33000,0


Let's check is there any NaN or null values in the dataset

In [30]:
df.isna().sum()

Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

Here in this dataset we'r having nominal categoruical data and we know that our model dosen,t understand any kind of categorical data . so we have to encode it . B'c the this nominal categoricl data is in the independent column so we can apply pandas or one hot encoding

In [32]:
k = pd.get_dummies(df["Gender"], drop_first = True)

In [33]:
k

,Male
0,1
1,1
2,0
3,0
4,1
...,...
395,0
396,1
397,0
398,1


In [36]:
dff = pd.concat([df.drop("Gender",axis = 1), k ], axis = 1)

In [37]:
dff

,Age,EstimatedSalary,Purchased,Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


Let's shuffle the columns

In [85]:
dff.columns

Index(['Age', 'EstimatedSalary', 'Male', 'Purchased'], dtype='object')

In [86]:
dff =dff[['Age', 'EstimatedSalary','Male', 'Purchased']]

In [87]:
dff

,Age,EstimatedSalary,Male,Purchased
0,19,19000,1,0
1,35,20000,1,0
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,1,0
...,...,...,...,...
395,46,41000,0,1
396,51,23000,1,1
397,50,20000,0,1
398,36,33000,1,0


Let's assign independent columns to "X" and dependet to "y"

In [88]:
X = dff.drop("Purchased", axis = 1)

In [89]:
X

,Age,EstimatedSalary,Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [90]:
y = dff["Purchased"]

In [91]:
y

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int64

## first going to apply SVM without sacling and without crossvalidation

In [116]:
from sklearn.model_selection import train_test_split

In [117]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X,y, train_size = 0.80)

In [118]:
Xtrain.shape, Xtest.shape , ytrain.shape , ytest.shape

((320, 3), (80, 3), (320,), (80,))

In [119]:
from sklearn.svm import SVC

In [120]:
sv = SVC()

In [121]:
sv.fit(Xtrain,ytrain)

SVC()

In [122]:
sv.score(Xtest,ytest)

0.7625

Let's do it with scaling

In [123]:
from sklearn.preprocessing import StandardScaler

In [124]:
ss = StandardScaler()

In [125]:
ss.fit(Xtrain)

StandardScaler()

In [126]:
Xtrain_ss =ss.transform(Xtrain)

In [127]:
Xtest_ss = ss.transform(Xtest)

once again applying SVM on scaled data 

In [128]:
sv1 = SVC()

In [129]:
sv1.fit(Xtrain_ss,ytrain)

SVC()

In [130]:
sv1.score(Xtest_ss,ytest)

0.9125

Let's do crossvalidation

In [131]:
from sklearn.model_selection import KFold,cross_val_score

In [132]:
kf = KFold(n_splits = 10 )

In [133]:
sss = StandardScaler()

In [134]:
sss.fit(X)

StandardScaler()

In [135]:
X_sss = sss.transform(X)

In [136]:
X_sss

array([[-1.78179743, -1.49004624,  1.02020406],
       [-0.25358736, -1.46068138,  1.02020406],
       [-1.11320552, -0.78528968, -0.98019606],
       ...,
       [ 1.17910958, -1.46068138, -0.98019606],
       [-0.15807423, -1.07893824,  1.02020406],
       [ 1.08359645, -0.99084367, -0.98019606]])

In [137]:
sv2= SVC()   # here defaut kernal is "rbf"

In [114]:
score = cross_val_score(sv2, X_sss,y,cv = kf)

In [115]:
score.mean()

0.9075

let's apply SVC again and again for every Kernal

In [140]:
sv3 =SVC(kernel = "linear")

In [141]:
sv4 =SVC(kernel = "poly")

In [145]:
sv5 = SVC(kernel = "sigmoid")

In [146]:
svclist = [sv3,sv4,sv5]

In [147]:
for model in svclist:
    score= cross_val_score(model,X_sss,y,cv=kf)
    print("the model is {} and the score is {}".format(model,score.mean()))

the model is SVC(kernel='linear') and the score is 0.8200000000000001
the model is SVC(kernel='poly') and the score is 0.8300000000000001
the model is SVC(kernel='sigmoid') and the score is 0.7975


Now we know that we are having some other hyper parameters such as "c" and "gamma" and this c and gamm is also having different -2values . It would be very time consuming process . so to get the best score a techanic is used i.e GridSearchCV  and  RandomSearchCv 

In [148]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

In [153]:
svg = SVC()

In [159]:
params = {"kernel" :["rbf","sigmoid","linear","poly"],
         "C":[1.0,5.0,8.0,100]}

In [160]:
grid = GridSearchCV(svg,param_grid=params, cv = 10)

In [161]:
grid.fit(X_sss,y)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid={'C': [1.0, 5.0, 8.0, 100],
                         'kernel': ['rbf', 'sigmoid', 'linear', 'poly']})

In [163]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008909,0.005932,0.003077,0.004983,1.0,rbf,"{'C': 1.0, 'kernel': 'rbf'}",1.000,0.850,0.900,0.975,1.000,0.850,0.825,0.775,0.900,0.975,0.9050,0.075664,1
1,0.008470,0.008648,0.004871,0.007335,1.0,sigmoid,"{'C': 1.0, 'kernel': 'sigmoid'}",0.850,0.750,0.875,0.875,0.875,0.800,0.850,0.675,0.675,0.650,0.7875,0.087500,13
2,0.007852,0.007852,0.001563,0.004690,1.0,linear,"{'C': 1.0, 'kernel': 'linear'}",0.675,0.675,0.975,0.950,1.000,0.850,0.800,0.775,0.825,0.675,0.8200,0.118216,9
3,0.010984,0.007123,0.000000,0.000000,1.0,poly,"{'C': 1.0, 'kernel': 'poly'}",0.700,0.700,0.950,0.950,1.000,0.875,0.825,0.800,0.825,0.825,0.8450,0.096047,8
4,0.006582,0.007494,0.003211,0.006424,5.0,rbf,"{'C': 5.0, 'kernel': 'rbf'}",1.000,0.825,0.900,0.975,1.000,0.850,0.825,0.800,0.900,0.950,0.9025,0.071981,2
5,0.012619,0.006312,0.001562,0.004687,5.0,sigmoid,"{'C': 5.0, 'kernel': 'sigmoid'}",0.850,0.750,0.825,0.725,0.900,0.825,0.875,0.675,0.550,0.525,0.7500,0.125000,14
6,0.015774,0.000280,0.000000,0.000000,5.0,linear,"{'C': 5.0, 'kernel': 'linear'}",0.675,0.675,0.975,0.950,1.000,0.850,0.800,0.775,0.825,0.675,0.8200,0.118216,9
7,0.007362,0.007739,0.003124,0.006249,5.0,poly,"{'C': 5.0, 'kernel': 'poly'}",0.725,0.675,0.950,0.975,1.000,0.850,0.825,0.800,0.850,0.850,0.8500,0.098742,5
8,0.008189,0.008090,0.001648,0.004731,8.0,rbf,"{'C': 8.0, 'kernel': 'rbf'}",1.000,0.800,0.900,0.975,1.000,0.850,0.825,0.800,0.900,0.950,0.9000,0.075000,3
9,0.010284,0.006339,0.002361,0.005021,8.0,sigmoid,"{'C': 8.0, 'kernel': 'sigmoid'}",0.850,0.750,0.825,0.725,0.900,0.825,0.875,0.675,0.550,0.525,0.7500,0.125000,14


In [164]:
grid.best_params_

{'C': 1.0, 'kernel': 'rbf'}

In [165]:
grid.best_score_

0.905